In [1]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from torch import tensor
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import random
import os
import matplotlib.pyplot as plt
#from numba import jit
import pickle
from scipy.interpolate import interp1d
from torch.utils.data import DataLoader, random_split
import torch
from torchsummary import summary
import seaborn as sns
import sys
import torch.nn.functional as F
import pywt
from sklearn.preprocessing import MinMaxScaler
from torch.cuda import FloatTensor

# Req for package
sys.path.append("../")
from SkinLearning.Utils.NN import train, test, DEVICE, getParameterLoss
from SkinLearning.Utils.Dataset import getDataset, getSplit
from SkinLearning.Utils.Plotting import plotParameterBars


torch.backends.cudnn.benchmark = True

In [11]:
# Remove one FC LAyer
class RNN_hidden(nn.Module):
    def __init__(self):
        super(RNN_hidden, self).__init__()
        self.conv1 = nn.Conv1d(2, 128, kernel_size=5, padding=1, bias=False)
        self.pool1 = nn.MaxPool1d(kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm1d(128)
        
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.conv3 = nn.Conv1d(256, 512, kernel_size=3, padding=1, bias=False)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm1d(512)
        
        self.rnn = nn.RNN(15, 256, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128 , 64),
            nn.ReLU(),
            nn.Linear(64, 6),   
        )

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        
        h0 = torch.zeros(1, batch_size, 256).to(x.device)
        x, h = self.rnn(x, h0)
        
        x = h[-1].reshape(batch_size, -1)
        x = self.fc(x)
        
        x = x.view(batch_size, 6)
        return x

In [12]:
# Remove one FC LAyer
class RNN_output(nn.Module):
    def __init__(self):
        super(RNN_output, self).__init__()
        self.conv1 = nn.Conv1d(2, 128, kernel_size=5, padding=1, bias=False)
        self.pool1 = nn.MaxPool1d(kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm1d(128)
        
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.conv3 = nn.Conv1d(256, 512, kernel_size=3, padding=1, bias=False)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm1d(512)
        
        self.rnn = nn.RNN(15, 256, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(256, 6),
        )

    def forward(self, x):
        a = x.shape
        batch_size = x.shape[0]
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        
        
        h0 = torch.zeros(1, batch_size, 256).to(x.device)
        out, h = self.rnn(x, h0)
  
        x = out[:, -1, :]
        x = x.reshape(batch_size, -1)

        
        x = self.fc(x)
        
        x = x.view(batch_size, 6)
        return x

In [13]:
# Remove one FC LAyer
class RNN_orig(nn.Module):
    def __init__(self):
        super(RNN_orig, self).__init__()
        self.conv1 = nn.Conv1d(2, 128, kernel_size=5, padding=1, bias=False)
        self.pool1 = nn.MaxPool1d(kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm1d(128)
        
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.conv3 = nn.Conv1d(256, 512, kernel_size=3, padding=1, bias=False)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm1d(512)
        
        self.rnn = nn.RNN(15, 256, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(131072, 1024),
            nn.ReLU(),
            nn.Linear(1024 , 512),
            nn.ReLU(),
            nn.Linear(512, 6),   
        )

    def forward(self, x):
        a = x.shape
        batch_size = x.shape[0]
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        
        
        h0 = torch.zeros(1, batch_size, 256).to(x.device)
        out, h = self.rnn(x, h0)
  
        x = out
        x = x.reshape(batch_size, -1)

        
        x = self.fc(x)
        
        x = x.view(batch_size, 6)
        return x

In [14]:
# Remove one FC LAyer
class RNN_both(nn.Module):
    def __init__(self):
        super(RNN_both, self).__init__()
        self.conv1 = nn.Conv1d(2, 128, kernel_size=5, padding=1, bias=False)
        self.pool1 = nn.MaxPool1d(kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm1d(128)
        
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.conv3 = nn.Conv1d(256, 512, kernel_size=3, padding=1, bias=False)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm1d(512)
        
        self.rnn = nn.RNN(15, 256, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256 , 128),
            nn.ReLU(),
            nn.Linear(128, 6),   
        )

    def forward(self, x):
        a = x.shape
        batch_size = x.shape[0]
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        
        
        h0 = torch.zeros(1, batch_size, 256).to(x.device)
        out, h = self.rnn(x, h0)
  
        x = torch.concat([h[-1], out[:, -1, :]])
        x = x.reshape(batch_size, -1)

        
        x = self.fc(x)
        
        x = x.view(batch_size, 6)
        return x

In [15]:
# Uses full hidden state
class RNN_fh(nn.Module):
    def __init__(self):
        super(RNN_fh, self).__init__()
        self.conv1 = nn.Conv1d(2, 128, kernel_size=5, padding=1, bias=False)
        self.pool1 = nn.MaxPool1d(kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm1d(128)
        
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.conv3 = nn.Conv1d(256, 512, kernel_size=3, padding=1, bias=False)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm1d(512)
        
        self.rnn = nn.RNN(15, 256, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128 , 64),
            nn.ReLU(),
            nn.Linear(64, 6),   
        )

    def forward(self, x):
        a = x.shape
        batch_size = x.shape[0]
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        
        
        h0 = torch.zeros(1, batch_size, 256).to(x.device)
        out, h = self.rnn(x, h0)
  
        x = h
        x = x.reshape(batch_size, -1)

        
        x = self.fc(x)
        
        x = x.view(batch_size, 6)
        return x

In [16]:
dataset, scaler = getDataset()

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:09<00:00, 246.42it/s]


In [17]:
train_loader, test_loader = getSplit(dataset)

In [21]:
out = RNN_output()

In [22]:
out_train_loss, out_val_loss =  train(train_loader, out, val_loader=test_loader, LR=0.0001, epochs=550)

Using: cuda:0


Epoch 68/550: 100%|████████████████████████████████| 56/56 [00:00<00:00, 77.33batch/s, lastLoss=0.0774, valLoss=0.0758]


Epoch 136/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 80.49batch/s, lastLoss=0.0676, valLoss=0.0668]


Epoch 204/550: 100%|████████████████████████████████| 56/56 [00:00<00:00, 78.63batch/s, lastLoss=0.0602, valLoss=0.056]


Epoch 272/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 79.10batch/s, lastLoss=0.0538, valLoss=0.0566]


...

Epoch 408/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 77.44batch/s, lastLoss=0.0456, valLoss=0.0492]


Epoch 476/550: 100%|████████████████████████████████| 56/56 [00:00<00:00, 77.63batch/s, lastLoss=0.0419, valLoss=0.042]


Epoch 544/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 77.83batch/s, lastLoss=0.0392, valLoss=0.0393]


Epoch 550/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 79.06batch/s, lastLoss=0.0401, valLoss=0.0419]


Average train loss: 0.058039054373859776
Average validation loss: 0.05829862871100054


In [25]:
def test(test_loader, net, scaler):
    net.to(DEVICE)
    net.eval()
    criterion = nn.L1Loss()

    losses = []
    p_losses = []
    mae = []
    orig = []

    with torch.no_grad():
        with tqdm(test_loader, unit=" batch") as it:
            for idx, data in enumerate(it):
                inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)

                predicted = net(inp)
                
                # Denormalise
                p = scaler.inverse_transform(predicted.cpu().numpy())
                o = scaler.inverse_transform(out.cpu().numpy())
                    
                # Get column wise and overall MAPE
                # Since each column is normalised should also be able to use MAE*100
                p_loss = np.mean(100*(np.abs(o-p)/o), axis=0)
                loss = np.mean(100*(np.abs(o-p)/o))

                curr_mae = criterion(predicted, out).item()
                mae.append(curr_mae)
        
            
                p_losses.append(p_loss)
                losses.append(loss)
                orig.append(curr_mae)

            
    average_mape = 100 - np.mean(losses)
    average_p_loss = 100 - np.mean(p_losses, axis=0)
    mae_mean = np.mean(mae)
    mape2 = 100 - np.mean(orig)
    
    return average_mape, average_p_loss, mae_mean, mape2

In [27]:
test(test_loader, out, scaler)

100%|█████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 147.08 batch/s]


(93.49792051315308,
 array([95.07151 , 88.322296, 99.78066 , 99.81861 , 83.50792 , 94.486534],
       dtype=float32),
 0.04341856713096301,
 99.95658143286904)

In [28]:
hidden = RNN_hidden()

In [29]:
hidden_train_loss, hidden_val_loss =  train(train_loader, hidden, val_loader=test_loader, LR=0.0001, epochs=550)

Using: cuda:0


Epoch 68/550: 100%|████████████████████████████████| 56/56 [00:00<00:00, 76.70batch/s, lastLoss=0.0704, valLoss=0.0676]


Epoch 136/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 76.60batch/s, lastLoss=0.0566, valLoss=0.0749]


Epoch 204/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 77.28batch/s, lastLoss=0.0498, valLoss=0.0484]


Epoch 272/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 76.43batch/s, lastLoss=0.0435, valLoss=0.0479]


...

Epoch 408/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 77.00batch/s, lastLoss=0.0407, valLoss=0.0416]


Epoch 476/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 76.97batch/s, lastLoss=0.0375, valLoss=0.0423]


Epoch 544/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 75.75batch/s, lastLoss=0.0335, valLoss=0.0344]


Epoch 550/550: 100%|████████████████████████████████| 56/56 [00:00<00:00, 75.38batch/s, lastLoss=0.0339, valLoss=0.044]


Average train loss: 0.05106937584231378
Average validation loss: 0.05266756271373367


In [30]:
orig = RNN_orig()

In [31]:
orig_train_loss, orig_val_loss =  train(train_loader, orig, val_loader=test_loader, LR=0.0001, epochs=550)

Using: cuda:0


Epoch 68/550: 100%|████████████████████████████████| 56/56 [00:02<00:00, 20.28batch/s, lastLoss=0.0594, valLoss=0.0569]


Epoch 136/550: 100%|███████████████████████████████| 56/56 [00:02<00:00, 20.13batch/s, lastLoss=0.0439, valLoss=0.0544]


Epoch 204/550: 100%|████████████████████████████████| 56/56 [00:02<00:00, 20.15batch/s, lastLoss=0.0317, valLoss=0.043]


Epoch 272/550: 100%|███████████████████████████████| 56/56 [00:02<00:00, 20.08batch/s, lastLoss=0.0289, valLoss=0.0418]


...

Epoch 408/550: 100%|███████████████████████████████| 56/56 [00:02<00:00, 20.20batch/s, lastLoss=0.0202, valLoss=0.0385]


Epoch 476/550: 100%|███████████████████████████████| 56/56 [00:02<00:00, 20.23batch/s, lastLoss=0.0195, valLoss=0.0371]


Epoch 544/550: 100%|█████████████████████████████████| 56/56 [00:02<00:00, 20.28batch/s, lastLoss=0.018, valLoss=0.039]


Epoch 550/550: 100%|███████████████████████████████| 56/56 [00:02<00:00, 20.30batch/s, lastLoss=0.0162, valLoss=0.0362]


Average train loss: 0.034347546356609214
Average validation loss: 0.046026647793693524


In [32]:
fh = RNN_fh()

In [33]:
fh_train_loss, fh_val_loss =  train(train_loader, fh, val_loader=test_loader, LR=0.0001, epochs=550)

Using: cuda:0


Epoch 68/550: 100%|████████████████████████████████| 56/56 [00:00<00:00, 76.30batch/s, lastLoss=0.0676, valLoss=0.0644]


Epoch 136/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 76.75batch/s, lastLoss=0.0561, valLoss=0.0546]


Epoch 204/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 75.54batch/s, lastLoss=0.0502, valLoss=0.0516]


Epoch 272/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 76.35batch/s, lastLoss=0.0479, valLoss=0.0484]


...

Epoch 408/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 76.68batch/s, lastLoss=0.0392, valLoss=0.0429]


Epoch 476/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 76.84batch/s, lastLoss=0.0352, valLoss=0.0396]


Epoch 544/550: 100%|███████████████████████████████| 56/56 [00:00<00:00, 76.01batch/s, lastLoss=0.0337, valLoss=0.0438]


Epoch 550/550: 100%|████████████████████████████████| 56/56 [00:00<00:00, 76.88batch/s, lastLoss=0.0349, valLoss=0.038]


Average train loss: 0.05092195909754555
Average validation loss: 0.05235694397427142


In [34]:
both = RNN_both()

In [35]:
both_train_loss, both_val_loss =  train(train_loader, both, val_loader=test_loader, LR=0.0001, epochs=550)

Using: cuda:0


Epoch 68/550: 100%|██████████████████████████████████| 56/56 [00:00<00:00, 77.27batch/s, lastLoss=0.185, valLoss=0.188]


Epoch 136/550: 100%|█████████████████████████████████| 56/56 [00:00<00:00, 76.51batch/s, lastLoss=0.184, valLoss=0.186]


Epoch 204/550: 100%|█████████████████████████████████| 56/56 [00:00<00:00, 76.36batch/s, lastLoss=0.184, valLoss=0.181]


Epoch 272/550: 100%|█████████████████████████████████| 56/56 [00:00<00:00, 76.77batch/s, lastLoss=0.184, valLoss=0.182]


...

Epoch 408/550: 100%|█████████████████████████████████| 56/56 [00:00<00:00, 76.46batch/s, lastLoss=0.184, valLoss=0.184]


Epoch 476/550: 100%|█████████████████████████████████| 56/56 [00:00<00:00, 77.26batch/s, lastLoss=0.184, valLoss=0.182]


Epoch 544/550: 100%|█████████████████████████████████| 56/56 [00:00<00:00, 76.78batch/s, lastLoss=0.184, valLoss=0.184]


Epoch 550/550: 100%|█████████████████████████████████| 56/56 [00:00<00:00, 76.55batch/s, lastLoss=0.184, valLoss=0.187]


Average train loss: 0.18457281245523458
Average validation loss: 0.18523676806462536


In [ ]:
df = getParameterLoss([out, hidden, orig, fh, both],["Output[-1]", "Hidden[-1]", "Original (output)", "Hidden", "Hidden[-1] + Output[-1]"])

In [36]:
models = [out, hidden, orig, fh, both]
train = [out_train_loss, hidden_train_loss, orig_train_loss, fh_train_loss, both_train_loss]
test = [out_val_loss, hidden_val_loss, orig_val_loss, fh_val_loss, both_val_loss]

data = [models, train, test]

In [39]:
import pickle

with open("../Results/RNNOut.pkl", "wb") as f:
    pickle.dump(data, f)

In [ ]:
plotParameterBars(df)

In [ ]:
sns.set_theme()

"""
    Plots train and test curves of given models
"""
def printCurves(names, train_loss, val_loss, epochs=400, name="train_test2"):
    if len(names)%2 == 0:
        rs = len(names)//2
        cs = len(names)//2
    else:
        rs, cs = len(names), 1
        
    fig, ax = plt.subplots(
        rs, cs, figsize=(35 if len(names) != 1 else 10, len(names)*6), sharex=True, sharey=True, constrained_layout=True)
    x = range(0, epochs)
    fig.supxlabel("Epoch", fontsize=45)
    fig.supylabel("MAE", fontsize=45)

    #ax[0][1].set_xlabel("Training Loss")
    ##ax[1][1].set_xlabel("Validation Loss")
    
    tv_loss = list(zip(train_loss, val_loss))
    if cs == 1 and rs == 1:
        ax.set_title(names[0], fontsize=40)
        ax.xaxis.set_tick_params(labelsize=35)
        ax.yaxis.set_tick_params(labelsize=35)
        for j in range(2):
            ax.plot(x, tv_loss[0][j], c = "b" if j == 0 else "y", label = "Train loss" if j == 0 else "Validation loss")
    elif cs == 1:
        for i in range(len(names)):
            ax[i].set_title(names[i], fontsize=40)
            ax[i].xaxis.set_tick_params(labelsize=35)
            ax[i].yaxis.set_tick_params(labelsize=35)
            for j in range(2):
                ax[i].plot(x, tv_loss[i][j], c = "b" if j == 0 else "y", label = "Train loss" if j == 0 else "Validation loss")
    else:
        pos = 0
        for i in range(rs):
            for j in range(cs):
                ax[i][j].set_title(names[pos], fontsize=40)
                ax[i][j].xaxis.set_tick_params(labelsize=35)
                ax[i][j].yaxis.set_tick_params(labelsize=35)
                for k in range(2):
                    ax[i][j].plot(x, tv_loss[pos][k], c = "b" if k == 0 else "y", label = "Train loss" if k == 0 else "Validation loss")
    
                pos += 1
        
    #plt.xticks(fontsize=14, rotation=90)
    plt.legend(loc='best', prop={'size':30})
    plt.show()
    fig.savefig(f"../Results/{name}.svg", bbox_inches='tight')